<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
%pip install nbstripout

REPO_URL="https://github.com/UH-Insure/Colab-Training.git"
REPO="Colab-Training"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

Repo 'Colab-Training' exists, pulling latest changes...
HEAD is now at 8051bea Move functions to module
Already up to date.
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/codex/incorporate-wandb-and-refactor-training-loop
  remotes/origin/main


In [2]:
from model.test import test
test()

Hello World!


In [3]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

Logged in as: j05hr3d


In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
#os.chdir("/content")
!pip install -q transformers datasets

import json, random
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer

In [6]:
!pip install -q transformers datasets peft flash-attn

MODEL = "Qwen/Qwen3-Coder-30B-A3B-Instruct"
DATASET = "/content/Colab-Training/MOC/all_hybrid.jsonl"
DATA_COLUMN = "content"

SEQ_LENGTH = 4096

# Training arguments
MAX_STEPS = 200
NUM_EPOCHS = 1
BATCH_SIZE = 12
GR_ACC_STEPS = 2
LR = 2e-4                     # learning_rate
WARMUP_RATIO = 0.03
WEIGHT_DECAY = 0.01
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.05  # weight_decay
NUM_WARMUP_STEPS = 100  # num_warmup_steps
EVAL_FREQ = 25
SAVE_FREQ = 50
LOG_FREQ = 10
OUTPUT_DIR = "peft-FT-3-Coder-30b-v2"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.25  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate


# LORA
LORA_R = 16  # lora_r
LORA_ALPHA = LORA_R * 2  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
#LORA_TARGET_MODULES = ["q_proj","k_proj","v_proj"]  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 42

In [7]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    TrainerCallback
)

set_seed(SEED)

In [8]:
import torch
from tqdm import tqdm
from model.dataset.util import chars_token_ratio, split_by_filetype

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL,
    trust_remote_code=True,
    use_fast=False,
)

train_ds, eval_ds = split_by_filetype(
    dataset_path=DATASET,
    seed=SEED,
    test_size=0.1,
    filetypes=["cry", "saw", "txt"]
)
print(train_ds[0].keys())
assert DATA_COLUMN in train_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"
print(eval_ds[0].keys())
assert DATA_COLUMN in eval_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"
print(len(train_ds), len(eval_ds))
chars_per_token = chars_token_ratio(train_ds, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

dict_keys(['filename', 'filetype', 'content', 'variant', 'set'])
dict_keys(['filename', 'filetype', 'content', 'variant', 'set'])
9 1
The character to token ratio of the dataset is: 3.11


In [9]:
from model.dataset.util import *

In [10]:
from model.dataset.constantLengthDataset import ConstantLengthDataset
import torch


train_dataset = ConstantLengthDataset(
        tokenizer=tokenizer,
        dataset=train_ds,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        overlap_ratio=0.125,
        seed=SEED,
        already_tokenized=False,
)
eval_dataset = ConstantLengthDataset(
        tokenizer=tokenizer,
        dataset=eval_ds,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        overlap_ratio=0.125,
        seed=SEED,
        already_tokenized=False,
)

[INFO] Empirical dataset length = 2 sequences (~6290 tokens)
[INFO] Empirical dataset length = 1 sequences (~555 tokens)


In [11]:
import os, gc, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect(); torch.cuda.empty_cache()

In [12]:
! pip install -U bitsandbytes

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

names = ["<|fim_prefix|>", "<|fim_middle|>", "<|fim_suffix|>", "<|fim_pad|>"]
ids = [tokenizer.convert_tokens_to_ids(t) for t in names]
print(dict(zip(names, ids)))
print("additional_special_tokens:", tokenizer.special_tokens_map.get("additional_special_tokens"))


# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

base = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=False,
        quantization_config=bnb_config,
        torch_dtype="float16",
        device_map="auto",
        use_cache=False,
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
)

base = prepare_model_for_kbit_training(base)

{'<|fim_prefix|>': 151659, '<|fim_middle|>': 151660, '<|fim_suffix|>': 151661, '<|fim_pad|>': 151662}
additional_special_tokens: ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

In [13]:
blk = base.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = LORA_TARGET_MODULES

ATTN: Qwen3MoeAttention(
  (q_proj): Linear4bit(in_features=2048, out_features=4096, bias=False)
  (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (o_proj): Linear4bit(in_features=4096, out_features=2048, bias=False)
  (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
  (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
)
MLP: Qwen3MoeSparseMoeBlock(
  (gate): Linear4bit(in_features=2048, out_features=128, bias=False)
  (experts): ModuleList(
    (0-127): 128 x Qwen3MoeMLP(
      (gate_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (up_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (down_proj): Linear4bit(in_features=768, out_features=2048, bias=False)
      (act_fn): SiLUActivation()
    )
  )
)


In [14]:
# Set up lora
from peft import PeftModel

peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

# model = PeftModel.from_pretrained(base, "tam2003/peft-FT-3-Coder-30b") # If your just trying to test the model.
model = get_peft_model(base, peft_config) # If you want to continue training.

model.print_trainable_parameters()

trainable params: 843,841,536 || all params: 31,375,964,160 || trainable%: 2.6895


In [15]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
class PerplexityCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        if state.log_history and "eval_loss" in state.log_history[-1]:
            eval_loss = state.log_history[-1]["eval_loss"]
            ppl = math.exp(eval_loss) if eval_loss < 20 else float("inf")
            print(f"\n>>> Epoch {int(state.epoch) if state.epoch is not None else '?'} | "
                  f"eval_loss={eval_loss:.4f} | perplexity={ppl:.3f}\n")


In [16]:
train_ds.start_iteration = 0

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=False,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_steps=4,
    #warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    fp16=FP16,
    bf16=BF16,
    optim="paged_adamw_8bit",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to=["tensorboard"],
    push_to_hub=True,
    dataloader_drop_last=False,
    dataloader_num_workers=2,
    include_tokens_per_second=True,
    max_grad_norm=0.3,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_pin_memory=True,
    save_safetensors=True,
    seed=SEED,
)

print(f"Training samples: {len(train_ds)}")
print(f"Sample text:\n\n{train_ds[0][DATA_COLUMN][:400]}")

Training samples: 9
Sample text:

x = [True, False]
y = [False, True]
z = x + y

property p1 = z == 3

xx = [[True,False]]
yy = [[False, True]]
zz = xx + yy



t : {a} [a*3] -> [a*3][a*3]
t d = zero



In [17]:
import math
from transformers import TrainerCallback
from time import time

class EpochToDatasetCallback(TrainerCallback):
    def __init__(self, dset):
        self.dset = dset
    def on_epoch_begin(self, args, state, control, **kwargs):
        if hasattr(self.dset, "set_epoch"):
            self.dset.set_epoch(state.epoch)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=collator,
    callbacks=[PerplexityCallback(), EpochToDatasetCallback(train_dataset)],
)
train_start_time = time()
print("Training...")
trainer.train(resume_from_checkpoint=False)
train_end_time = time()
print(f"Training completed in {train_end_time - train_start_time:.2f} seconds.")
print(f"Total examples trained: {train_ds[0].keys()}")
eval_results = trainer.evaluate()
eval_loss = eval_results["eval_loss"]
perplexity = math.exp(eval_loss)
print(f"Eval loss = {eval_loss:.2f}, Perplexity = {perplexity:.2f}")

/tmp/ipython-input-1930375221.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 151643}.


Training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Casting fp32 inputs back to torch.bfloat16 for flash-attn compatibility.


Epoch,Training Loss,Validation Loss
1,1.017300,0.911081


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:751: UserWarning: Length of IterableDataset <model.dataset.constantLengthDataset.ConstantLengthDataset object at 0x7bd30dd105f0> was reported to be 1(when accessing len(dataloader)), but 2 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)



>>> Epoch 1 | eval_loss=0.9111 | perplexity=2.487

Training completed in 204.21 seconds.
Total examples trained: dict_keys(['filename', 'filetype', 'content', 'variant', 'set'])


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:751: UserWarning: Length of IterableDataset <model.dataset.constantLengthDataset.ConstantLengthDataset object at 0x7bd30dd105f0> was reported to be 1(when accessing len(dataloader)), but 2 samples have been fetched. For multiprocessing data-loading, this could be caused by not properly configuring the IterableDataset replica at each worker. Please see https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset for examples.
  warnings.warn(warn_msg)



>>> Epoch 1 | eval_loss=0.9111 | perplexity=2.487

Eval loss = 0.91, Perplexity = 2.49


In [18]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...-30b-v2/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...8741.6c6cf53d4df0.12002.0: 100%|##########| 6.81kB / 6.81kB            

  ...18274.6c6cf53d4df0.7627.1: 100%|##########|   354B /   354B            

  ...18081.6c6cf53d4df0.7627.0: 100%|##########| 6.81kB / 6.81kB            

  ...adapter_model.safetensors:   1%|1         | 41.9MB / 3.38GB            

  ...8933.6c6cf53d4df0.12002.1:  76%|#######5  |   268B /   354B            

CommitInfo(commit_url='https://huggingface.co/j05hr3d/peft-FT-3-Coder-30b-v2/commit/294b4f3a3016d8d73451b3c75dd5ccca8bad5f5f', commit_message='End of training', commit_description='', oid='294b4f3a3016d8d73451b3c75dd5ccca8bad5f5f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/j05hr3d/peft-FT-3-Coder-30b-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='j05hr3d/peft-FT-3-Coder-30b-v2'), pr_revision=None, pr_num=None)